# Changing to Logspace
Switching to logspace for the mass and luminosity distances for the fitting. Hopefully this should produce a better fit since now the KDE estimates cannot go into negative galues in logspace and the uncertainty scales better with the means

In [1]:
import numpy as np
from scipy.stats import gaussian_kde
import pymc as pm
from pylab import *
import arviz as az
from astropy.cosmology import Planck18
from tqdm import tqdm
import seaborn as sns
from MCMC_functions import *
from astropy.cosmology import units as cu

sns.set_context('notebook')
sns.set_palette('colorblind')

import matplotlib
matplotlib.rcParams['font.family'] = 'serif' # so these plots can go into thesis


%matplotlib inline